## Todo
ANALYSIS
- longest listening session
- PLAYLISTS (this will be a little tougher?)
- GENRE ANALYSIS FUNCTIONS

NEW FEATURES
- matplotlib & GUI libraries (until more aesthetic things can be found) (**WIP**)
- skip detection via timestamps of listen & duration of song
- combine songs with identical titles and artists but different URLs (under the same URL - idfk). Create ID system instead of using URLs as identifiers).
	- maybe have a google photos "is this the same person" style prompt for these once a UI is created
- databse that doesn't force you to ping API 4K times per analysis if you just want to add a couple songs
- add musicbrainz (or another) API to get genre and decade and album and other really good song data (**WIP**)

GENERAL USABILITY
- make all calculations involving durations optional depending on the durations flag (from the YT api)
	- maybe even remove flags all together and just make a GUI because there are going to be so many options
- this program is REALLY bad at accounting for everything
  - from leap years
  - to days not listened to music
  - to if the user doesn't have the durations
  - yeah im sure theres so much other stuff
  - new features are EST-only friendly (timezone flag could fix?)
- primitive documentation
- fix flags system (mostly for debugging at this pt)
- add stuff to the actual html report

LONG TERM
- understand songs' relation to one another by their sequence
- music as NLP: each song is a word, each session a sentence
	- start by BoW model using song IDs

In [ ]:
import datetime
import sys
import getopt
import json
import requests
import re
import pandas as pd
import collections
import os
import itertools
import matplotlib.pyplot as plt
import pytz
import itertools
import numpy as np
import urllib.parse
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import HTML, display
import time
import ast

In [ ]:
#drive.mount('/content/drive', force_remount=True)

### Loader class

In [ ]:
class Loader():
    # backbone methods
    def __init__(self, duration, more_details, use_songs, analyze_year, apikey, filepath, ignores={"Title": [], "Artist": [], "URL": []}):
        self.duration, self.more_details, self.use_songs, self.analyze_year, self.apikey, self.filepath, self.ignores = (duration, more_details, use_songs, analyze_year, apikey, filepath, ignores)
        self.file = self.open_file(self.filepath, ".json")
        self.out = display(self.progress(0, 100), display_id=True)

    # utility methods
    def should_not_ignore(self, title, year, header):
        if (header == "YouTube Music" and title[:7] == "Watched" and year[:4] in self.analyze_year):
            return True
        return False

    def open_file(self, filepath, filetype):
        if (filepath.endswith(filetype)):
            try:
                file = open(filepath, "r", encoding="utf8")
                return file
            except:
                print("Could not open your report file")
                sys.exit()
        else:
            print("There was an error opening your report files.")
            sys.exit()


    # processes bulk lists
    def parse_json(self):
        self.history = {"Title": [], "Artist": [], "Year": [], "URL": [], "Duration": []}
        json_object = json.load(self.file)
        for obj in json_object:
            if (self.should_not_ignore(obj['title'], obj['time'], obj['header'])):
                if ('subtitles' in obj):
                    self.history["Title"].append(obj['title'][8:])
                    self.history["Artist"].append(obj['subtitles'][0]['name'].replace('- Topic ', '').replace('- Topic', ''))
                    self.history["Year"].append(obj['time'])
                    self.history["URL"].append(obj['titleUrl'][32:])
                    self.history["Duration"].append(0)

        # preprocessing
        for i in range(len(self.history["Title"])):
            if i >= len(self.history["Title"]):
                break
            if self.history["Title"][i] in self.ignores["Title"] or self.history["Artist"][i] in self.ignores["Artist"] or self.history["URL"][i] in self.ignores["URL"]:
                del self.history["Title"][i]
                del self.history["Artist"][i]
                del self.history["Year"][i]
                del self.history["URL"][i]
                del self.history["Duration"][i]
                i-= 1

        occurrences = collections.Counter(self.history['URL'])
        self.history['Occurrences'] = []
        for i in self.history['URL']:
            self.history['Occurrences'].append(occurrences[i])

        occurrences = collections.Counter(self.history['Artist'])
        duration = [0]*len(occurrences.keys())
        self.artists = {"Artist": occurrences.keys(), "Occurrences": occurrences.values(), "Duration": duration}

    # generates dataframes and csv files
    def gen_csvs(self, genfolder=True):
        if genfolder:
            os.chdir("generated-reports") # may be volatile
            datestring = "-" + str(datetime.datetime.now().strftime("%d.%m.%Y-%H.%M.%S"))
            os.mkdir(datestring[1:])
            os.chdir(datestring[1:])
        else:
            datestring = ""

        self.historyDF = pd.DataFrame(self.history)
        self.historyDF.to_csv("report-history.csv")

        self.artistsDF = pd.DataFrame(self.artists)
        self.artistsDF.to_csv("report-artists.csv")

        self.songsDF = pd.DataFrame(self.history)
        self.total_songs = len(self.songsDF)
        # TODO more comprehensive duplicate dropping function (much further down the line)
        self.songsDF.drop_duplicates(subset=['URL'], inplace=True)
        self.unique_songs = len(self.songsDF)
        self.songsDF = self.songsDF.reset_index(drop=True)
        self.songsDF.to_csv("report-songs.csv")

        if genfolder:
            os.chdir("../../")

    # API management functions
    def parse_duration(self, duration):
        timestr = duration
        time = re.findall(r'\d+', timestr)
        length = len(time)
        if length > 4:
            return 0
        if length == 4:
            return ((int(time[0])*24*60*60)+(int(time[1])*60*60)+int(time[2]*60)+(int(time[3])))
        elif length == 3:
            return ((int(time[0])*60*60)+(int(time[1])*60)+(int(time[2])))
        elif length == 2:
            return ((int(time[0])*60)+(int(time[1])))
        elif length == 1:
            return (int(time[0]))
        else:
            return 0

    def call_api(self, idlist):
        parameters = {"part": "contentDetails,snippet",
                      "id": ','.join(idlist), "key": self.apikey}
        response = requests.get(
            "https://www.googleapis.com/youtube/v3/videos", params=parameters)

        if (response.status_code == 200):
            json_parsed = response.json()
            for item in json_parsed['items']:
                duration = self.parse_duration(item['contentDetails']['duration'])
                url = item['id']
                if duration < 10:
                    duration = duration * 60

                # update by url
                for (j, i) in enumerate(self.history["URL"]):
                    if i == url:
                        if duration >= 10:
                            self.history["Duration"][j] = duration

    def progress(self, value, max=100):
        return HTML("""
            <progress
                value='{value}'
                max='{max}',
                style='width: 100%'
            >
                {value}
            </progress>
        """.format(value=value, max=max))

    def get_duration(self):
        # Count duration
        idlist = []
        calls = 0
        unique_song_urls = set(self.history['URL'])
        len_usurl = len(unique_song_urls)
        print("Getting durations. This may take a while. Awaiting", len_usurl, "requests.")
        for url in unique_song_urls:
            idlist.append(url)
            if len(idlist) == 50:
                # print("\tGetting info on videos " + str(1+50*calls) + " - " + str(50+50*calls)) # uglier GUI
                self.out.update(self.progress(((1+50*calls)*100)/len_usurl, 100))
                self.call_api(idlist)
                calls += 1
                idlist = []
        # print("\tGetting info on videos " + str(1+50*calls) + " - " + str(len(unique_song_urls))) # uglier GUI
        self.out.update(self.progress(100, 100))
        self.call_api(idlist)

        # update artist durations
        # i know this section is garbage i'll make it nicer later
        artist_durations = {}
        for i in range(len(self.history["Artist"])):
            duration = self.history["Duration"][i]
            artist = self.history["Artist"][i]
            try:
                artist_durations[artist] += duration
            except:
                artist_durations[artist] = duration

        occurrences = collections.Counter(self.history["Artist"])
        artists_dict = collections.defaultdict(list)
        for i in (artist_durations, occurrences):
            for key, val in i.items():
                artists_dict[key].append(val)

        durations = []
        occurrences = []
        for i, j in artists_dict.values():
            durations.append(i)
            occurrences.append(j)

        self.artists = {"Artist": artists_dict.keys(), "Occurrences": occurrences, "Duration": durations}

        self.gen_csvs()

    def outs(self):
        print("We are now processing your file")
        self.parse_json()

        if self.duration:
            self.get_duration()
        else:
            self.gen_csvs() # generates dataframes and CSVs

        return self.historyDF, self.artistsDF, self.songsDF

    def load(self, loadfp):
        print("Loading your preprocessed history files")
        historyDF = pd.read_csv(self.open_file(os.path.join(loadfp, "report-history.csv"), ".csv"))
        artistsDF = pd.read_csv(self.open_file(os.path.join(loadfp, "report-artists.csv"), ".csv"))
        songsDF = pd.read_csv(self.open_file(os.path.join(loadfp, "report-songs.csv"), ".csv"))
        return historyDF, artistsDF, songsDF

### Analyzer class

In [ ]:
class Analyzer():
    def __init__(self, historyDF, artistsDF, songsDF):
        self.history = historyDF
        self.artists = artistsDF
        self.songs = songsDF

    def tops(self):
        # Top 10 Songs
        songs_top = self.songs.nlargest(10, ['Occurrences'])

        # Top 10 Artists
        artists_top = self.artists.nlargest(10, ['Duration'])

        return artists_top, songs_top

    def meta(self):
        meta = {}
        meta["Total Seconds"] = sum(self.history["Duration"])
        meta["Total Songs"] = len(self.history["Title"])
        meta["Unique Songs"] = len(self.songs["Title"])
        meta["Unique Artists"] = len(self.artists["Artist"])
        return meta

    # specific optional analysis functions
    def uniques(self):
        uniques = {}
        uniques["Top 10 Unique Artists"] = collections.Counter(self.songs["Artist"]).most_common(10)
        return uniques

    def repeats(self):
        repeats = {}
        grouped_history = [(_, sum(1 for ii in i)) for _,i in itertools.groupby(self.history["URL"])]
        grouped_history.sort(key = lambda x : x[1])
        grouped_history.reverse()
        repeats["Most Consecutively Repeated Song"] = grouped_history[:10]
        repeats["Sweet 16"] = self.songs.nlargest(16, ["Occurrences"])
        repeats["Sweet 16 Artists Occurrences"] = self.artists.nlargest(16, ["Occurrences"])
        repeats["Sweet 16 Artists Durations"] = self.artists.nlargest(16, ["Duration"])

        return repeats

    def chronology(self, n=3):
        # this stuff will break if you are analyzing a period longer than a year

        chronology = {}
        top_songs_per_month = []
        for month in range(12):
            songs_for_month = {"Title": [], "Artist": [], "URL": [], "Duration": []}
            for j,i in enumerate(self.history["Year"]):
                if int(i[5:7]) == month+1:
                    songs_for_month["Title"].append(self.history["Title"][j])
                    songs_for_month["Artist"].append(self.history["Artist"][j])
                    songs_for_month["URL"].append(self.history["URL"][j])
                    songs_for_month["Duration"].append(self.history["Duration"][j])

            occurrences = collections.Counter(songs_for_month['URL'])
            songs_for_month['Occurrences'] = []
            for i in songs_for_month['URL']:
                songs_for_month['Occurrences'].append(occurrences[i])
            songs_for_month_DF = pd.DataFrame(songs_for_month)
            songs_for_month_DF.drop_duplicates(subset=['URL'], inplace=True)
            top_songs_per_month.append(songs_for_month_DF.nlargest(n, ['Occurrences']))

        chronology["Top n Songs Per Month"] = top_songs_per_month

        days = {}
        for j, i in enumerate(self.history["Year"]):
            try:
                days[i[5:10]]["Title"].append(self.history["Title"][j])
                days[i[5:10]]["Artist"].append(self.history["Artist"][j])
                days[i[5:10]]["URL"].append(self.history["URL"][j])
                days[i[5:10]]["Duration"].append(self.history["Duration"][j])
            except:
                songs_for_day = {"Title": [self.history["Title"][j]], "Artist": [self.history["Artist"][j]], "URL": [self.history["URL"][j]], "Duration": [self.history["Duration"][j]]}
                days[i[5:10]] = songs_for_day

        """
        # TEMPORARILY BLOCKED BECAUSE ITS SUPER BROKEN

        day_most_listened = -1 # day you listened to the most music function
        durations_per_day = [] # durations per day function
        songs_listened_dml = -1
        for j, i in enumerate(days.values()):
            urls = collections.Counter(i["URL"])
            if ((len(urls.values())) > songs_listened_dml):
                day_most_listened = j
                songs_listened_dml = len(urls.values())
            durations_per_day.append(sum(i["Duration"]))

        # since it counts from NOW to the past, this is in reverse order (this breaks of day 1 of the dataset isn't jan 1... uh oh)
        chronology["Most Diverse Day"] = [len(days) - day_most_listened , durations_per_day[day_most_listened]//60, songs_listened_dml] # this is the Nth day of the year
        chronology["Durations Per Day"] = durations_per_day
        chronology["Most Musical Day"] = [len(days) - durations_per_day.index(max(durations_per_day)), max(durations_per_day)//60]
        """

        times = {}
        localtime = pytz.timezone("US/Eastern")
        hrs = [i for i in range(0, 24)]
        errs = 0
        chronology["Songs Per Time of Day"] = dict()
        for i in range(24):
            chronology["Songs Per Time of Day"][i] = 0
        for i in self.history["Year"]:
            try:
                isDST = bool(localtime.localize(datetime.datetime.strptime(i[:-5], "%Y-%m-%dT%H:%M:%S")).dst())
                x = int(i[11:13])
                if isDST:
                    x += 1
                x -= 5
                x = hrs[x]
                try:
                    times[x] += 1
                except:
                    times[x] = 1
            except:
                errs += 1

        for i, j in times.items():
            chronology["Songs Per Time of Day"][i] = j
        chronology["Songs Per Time of Day Errors"] = errs

        weekdays = {}
        errs = 0
        for i in self.history["Year"]:
            try:
                wd = datetime.datetime.strptime(i[:-5], "%Y-%m-%dT%H:%M:%S").weekday()
                try:
                    weekdays[wd] += 1
                except:
                    weekdays[wd] = 1
            except:
                errs += 1

        chronology["Days of the Week"] = weekdays
        chronology["Days of the Week Errors"] = errs

        # this code can technically be inserted earlier when we process months
        months = {}
        for i in self.history["Year"]:
            try:
                months[int(i[5:7])] += 1
            except:
                months[int(i[5:7])] = 1

        chronology["Songs Per Month"] = months

        return chronology

    def averages(self):
        averages = {}
        averages["Average Song Length"] = sum(self.history["Duration"]) / len(self.history["Duration"])
        averages["Average Song Length Unique"] = sum(self.songs["Duration"]) / len(self.songs["Duration"])
        years = []
        for i in self.history["Year"]:
            years.append(i[5:10])
        averages["Average Seconds per Day"] = sum(self.history["Duration"]) / len(collections.Counter(years))

        min_song_length = min(self.songs["Duration"])
        max_song_length = max(self.songs["Duration"])
        min_song_idx = list(self.songs["Duration"]).index(min_song_length)
        max_song_idx = list(self.songs["Duration"]).index(max_song_length)

        averages["Shortest Song"] = [min_song_length, self.songs["Title"][min_song_idx], self.songs["Artist"][min_song_idx]]
        averages["Longest Song"] = [max_song_idx, self.songs["Title"][max_song_idx], self.songs["Artist"][max_song_idx]]

        # 5th percentile song by duration?
        # 95th percentile song by duration?
        # median song length

        history_duration_sorted = self.history["Duration"].copy()
        songs_duration_sorted = self.songs["Duration"].copy()
        history_duration_sorted = list(history_duration_sorted)
        songs_duration_sorted = list(songs_duration_sorted)
        history_duration_sorted.sort()
        songs_duration_sorted.sort()
        averages["Median Song Length"] = history_duration_sorted[int(len(history_duration_sorted)/2)]
        averages["Median Song Length Unique"] = songs_duration_sorted[int(len(songs_duration_sorted)/2)]

        #plt.hist(list(self.songs["Duration"]), 30, (0, 600))
        #plt.show()

        averages["Average Replays"] = sum(self.songs["Occurrences"]) / len(self.songs["Occurrences"])
        averages["Max Replays"] = max(self.songs["Occurrences"]) # yes this statistic is already calculated somewhere else

        frqtable = collections.Counter(self.songs["Occurrences"])

        averages["Occurrences Frequency Table"] = frqtable

        return averages

    def genres(self):
        # genre stuff!
        allowed_vals_f = open('genres.txt', 'r')
        allowed_vals = []
        for i in allowed_vals_f:
            allowed_vals.append(i.replace("\n", ""))

        artist_tags = {}
        print(len(self.artists['Artist']), "Artists")
        for c, i in enumerate(self.artists['Artist']):
            artist_tags[i] = []
            response = requests.get("https://musicbrainz.org/ws/2/artist/?fmt=json&query=name:" + urllib.parse.quote(i.strip()))
            if c % 20 == 0:
                print("Got", c, "artists")
            try:
                tags = response.json()['artists'][0]['tags']
                #print(tags)
                for ii in tags:
                    if ii['name'] in allowed_vals:
                        for iii in range(ii['count']):
                            artist_tags[i].append(ii['name'])
            except:
                #print("No tags for artist: " + i)
                pass

    def sessions(self):
        # split history into sessions (need to decide what distinguishes 1 session from another, most likely 30mins)
        # get longest session as a statistic
        # use sessions as "sentences" for starting to model music listening
        pass

### Formatter class

In [ ]:
class Formatter():
    def gen_report(self, args, meta, artists_top, songs_top):

        analyze_year, duration, more_details = args
        total_seconds = meta["Total Seconds"]

        # TODO rewrite HTML report generation
        # TODO Calculate total duration
        htmlreport = open('report.html', 'w', encoding=("utf8"))
        print("""<!DOCTYPE html><html><head><title>Wrapped</title><style type="text/css">body{background-color: #000000;}.center-div{position: absolute; margin: auto; top: 0; right: 0; bottom: 0; left: 0; width: 50%; height: 90%; background-color: #000000; border-radius: 3px; padding: 10px;}.ytm_logo{width: 15%;position: relative;top: 30px;left: 40px;}.title_logo{width: 30%;position: relative;top: 30px;left: 60px;}.right_title{position: absolute;font-family: "Product Sans";top: 55px;right: 10%;font-size: 2em;color: #ffffff;}.container{position: relative;top: 13%;left: 53px;}.minutes_title{font-family: "Product Sans";font-size: 2em;color: #ffffff;}.minutes{font-family: "Product Sans";font-size: 6em;color: #ffffff;}.row{display: flex;}.column{flex: 50%;}.list{font-family: "Roboto";font-size: 1.5em;line-height: 30px;color: #ffffff;}</style></head><body><div class="center-div"><img src="ytm_logo.png" class="ytm_logo"><img src="title.png" class="title_logo"/><span class="right_title">""", file=htmlreport)
        print(str(analyze_year), file=htmlreport)
        print(""" Wrapped</span><div class="container"><div class="minutes_title">Minutes Listened</div><div class="minutes">""", file=htmlreport)
        if duration:
            print(str(total_seconds//60), file=htmlreport)
        else:
            print("N/A", file=htmlreport)
        print("""</div><br><br><div class="row"><div class="column"><div class="minutes_title">Top Artists</div><div class="list">""", file=htmlreport)

        for i, j, v in zip(artists_top["Artist"], artists_top["Occurrences"], artists_top["Duration"]): # TODO add durations to artist stuff
            print("<br>", file=htmlreport)
            if more_details:
                if duration:
                    print('{0} - {1} songs ({2} mins)'.format(str(i), j, str(v//60)), file=htmlreport) # TODO v should be artist duration (?)
                    pass
                else:
                    print('{0} - {1} songs'.format(str(i), j), file=htmlreport)
            else:
                print('{0}'.format(i), file=htmlreport)
        print("""</div></div><div class="column"><div class="minutes_title">Top Songs</div><div class="list">""", file=htmlreport)
        top_songs, top_artists, top_occurrences = songs_top['Title'], songs_top['Artist'], songs_top['Occurrences']
        for i, j, k in zip(top_songs, top_artists, top_occurrences):
            print("<br>", file=htmlreport)
            if more_details:
                print('{0} - {1} - {2} plays'.format(j, i, k), file=htmlreport)
            else:
                print('{0} - {1}'.format(j, i), file=htmlreport)
        print("""</div></div></div></div></div></body></html>""", file=htmlreport)
        htmlreport.close()

### Execution code

In [ ]:
os.chdir("drive/My Drive/Colab Notebooks/ytmwrapped")

In [ ]:
# flags
# TODO fix parameter/flags system
more_details, duration, analyze_year, use_songs, apikey, load, loadfp = (True, True, ["2023"], True, None, True, os.getcwd()+"/generated-reports/21.08.2023-22.31.09")
token = open("apitoken.txt", "r").read()
apikey = token

ignore = {}
ignore["Title"] = open("ignore-title.txt", "r").read().split(",")
ignore["Artist"] = open("ignore-artist.txt", "r").read().split(",")
ignore["URL"] = open("ignore-url.txt", "r").read().split(",")
# filepath = sys.argv[1]
filepath = "watch-history.json"
loader = Loader(duration, more_details, use_songs, analyze_year, apikey, filepath, ignores=ignore)
if load:
    history, artists, songs = loader.load(loadfp)
else:
    history, artists, songs = loader.outs()

print()
print(history.head())
print()

analyzer = Analyzer(history, artists, songs)
artists_top, songs_top = analyzer.tops()
meta = analyzer.meta()

# TODO update formatter filesave system DO BEFORE UNCHECKING
#formatter = Formatter()
#formatter.gen_report((analyze_year, duration, more_details), meta, artists_top, songs_top)

print(" -- ANALYSIS DEBUGGING -- ")
print(" - Meta - ")
print(analyzer.meta())
print(" - Uniques - ")
print(analyzer.uniques())
print(" - Repeats - ")
#print(analyzer.repeats())
print(" - Chronology - ")
chrono = analyzer.chronology(n=5)
for j, i in enumerate(chrono["Top n Songs Per Month"]):
    print(j)
    print(i)
    pass
print(list(chrono.items())[1])
print(list(chrono.items())[3])
print(" - Averages - ")
averages = analyzer.averages()
print(averages)

print("All done!")

In [ ]:
# songs per times of the day
# parts of this code needs credit

times = chrono["Songs Per Time of Day"]
x = sorted(times.keys())
y = []
for i in x:
    y.append(times[i])

x = list(reversed(x))
y = list(reversed(y))

def rightRotate(lists, num):
    output_list = []

    for item in range(len(lists) - num, len(lists)):
        output_list.append(lists[item])

    for item in range(0, len(lists) - num):
        output_list.append(lists[item])

    return output_list

x = rightRotate(x, 6)
y = rightRotate(y, 6)

plt.figure(figsize=(20,10))
ax = plt.subplot(111, polar=True)
plt.axis('off')

upperLimit = max(y)
lowerLimit = min(y)

maxval = max(y)

slope = (maxval - lowerLimit) / maxval
heights = [slope * i + lowerLimit for i in y]

width = 2*np.pi / 24

indexes = list(range(1, 25))
angles = [element * width for element in indexes]

grey_heights = [slope*maxval + lowerLimit] * 24

# Draw bars
bars = ax.bar(
    x=angles,
    height=grey_heights,
    width=width,
    bottom=lowerLimit,
    linewidth=2,
    edgecolor="white",
    color="#d3d3d3",
)

bars = ax.bar(
    x=angles,
    height=heights,
    width=width,
    bottom=lowerLimit,
    linewidth=2,
    edgecolor="white",
    color="#61a4b2",
)

labelPadding = 5

for bar, angle, height, label in zip(bars,angles, heights, [str(i) for i in x]):

    rotation = np.rad2deg(angle)

    alignment = ""
    if angle >= np.pi/2 and angle < 3*np.pi/2:
        alignment = "right"
        rotation = rotation + 180
    else:
        alignment = "left"

    if (height > 100):
        ax.text(
            x=angle,
            y=lowerLimit + height + labelPadding,
            s=label,
            ha=alignment,
            va='center',
            rotation=rotation,
            rotation_mode="anchor")

In [ ]:
# listens per day of the week
weekdays = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
plt.bar(weekdays, chrono["Days of the Week"].values())
print(chrono["Days of the Week Errors"])

In [ ]:
# listens per month
plt.bar(chrono["Songs Per Month"].keys(), chrono["Songs Per Month"].values())

In [ ]:
plt.bar(averages['Occurrences Frequency Table'].keys(), averages['Occurrences Frequency Table'].values())
plt.yscale("linear") # "log"

In [ ]:
#drive.flush_and_unmount()

# MusicBrainz API

## Genres

In [ ]:
allowed_vals_f = open('genres.txt', 'r')
allowed_vals = []
for i in allowed_vals_f:
    allowed_vals.append(i.replace("\n", ""))

In [ ]:
artist_tags = {}
print(len(artists['Artist']), "Artists")
for c, i in enumerate(artists['Artist']):
    artist_tags[i] = []
    response = requests.get("https://musicbrainz.org/ws/2/artist/?fmt=json&query=name:" + urllib.parse.quote(i.strip()))
    if c % 20 == 0:
        print("Got", c, "artists")
    try:
        tags = response.json()['artists'][0]['tags']
        #print(tags)
        for ii in tags:
            if ii['name'] in allowed_vals:
                artist_tags[i].append(ii['name'])
    except:
        #print("No tags for artist: " + i)
        pass

In [ ]:
alltags = itertools.chain(*artist_tags.values())
counter = collections.Counter(alltags).most_common(10)
genres = []
counts = []
for (i, j) in counter:
    genres.append(i)
    counts.append(j)
plt.pie(counts, labels=genres, autopct="%1.1f%%")
plt.show()

In [ ]:
alltags = []
artist_tags = tags2
for i in history['Artist']:
    if isinstance(artist_tags[i], str):
        alltags.append(artist_tags[i])
    else:
        for ii in artist_tags[i]:
            alltags.append(ii)
counter = collections.Counter(alltags).most_common(10)
genres = []
counts = []
for (i, j) in counter:
    genres.append(i)
    counts.append(j)
plt.pie(counts, labels=genres, autopct="%1.1f%%")
plt.show()

In [ ]:
for i, j in tags2.items():
    if j == []:
        print(i)
        tags2[i] = input()

In [ ]:
with open('artist_tags.txt', 'w') as convert_file:
     convert_file.write(json.dumps(artist_tags))

In [ ]:
with open('artist_tags.txt', 'r') as convert_file:
     artist_tags = ast.literal_eval(convert_file.read())

## Year

In [ ]:
# just getting genre by genre of artist, not release (because release definitely has some issues)
arid = {}
ardate = {}
count = {}
for c, i in enumerate(artists['Artist']):
    i = i.strip()
    if c%40 == 0:
        print("Got", c, "artists")
    try:
        response = requests.get("https://musicbrainz.org/ws/2/artist/?fmt=json&query=name:" + urllib.parse.quote(i))
        arid[i] = response.json()['artists'][0]['id']
        # store the date in the thing
        response = requests.get("https://musicbrainz.org/ws/2/release-group/?fmt=json&query=arid:" + arid[i])
        frd = int(response.json()['release-groups'][0]['first-release-date'][0:4])
        for ii in response.json()['release-groups']:
            try:
                t = ii['first-release-date']
                if int(t[0:4]) < frd:
                    frd = int(t[0:4])
            except:
                continue
        ardate[i] = frd
    except:
        pass

In [ ]:
count = {}
for i in history['Artist']:
    i = i.strip()
    if i not in ardate.keys():
        continue
    if ardate[i] in count.keys():
        count[ardate[i]] += 1
    else:
        count[ardate[i]] = 1

In [ ]:
plt.bar(count.keys(), count.values())